# EDA

In [53]:
import pandas as pd

# Had to "pip install openpyxl"

dataset = pd.read_excel("pr11_credit_scores.xlsx")


# Things to do

- Essential to understand every variable of the dataset
- Check whether the columns are the right type
    - If no, force the correct type.
- After knowing data:
    - Plot and and see summary statistics
    - Check the unique values of each
    - Check if the data make makes sense
        - If not, see the most correct way to deal with it
- Check multicollinearity? How important is it when you would run a more advanced model?
- Check class imbalance in target variable. Is it necessary to do any sort of balancing?
- Would it be good to apply feature enginnering?
- Remove duplicates

In [54]:
dataset.describe(include="all")

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
count,100000,100000,100000,90015,99909.0,99909,99909,99909,84920.000000,99909.000000,...,100000,100000,100000.000000,90970,100000,100000.000000,95521,100000,98800,100000
unique,100000,12500,44,10139,1799.0,12501,31,18936,NaN,NaN,...,4,13178,NaN,404,3,NaN,91049,13,98792,3
top,0x1602,CUS_0xd40,August,Langep,38.0,#F%$D@*&8,_______,17273.83,NaN,NaN,...,Standard,1360.45,NaN,15 Years and 11 Months,Yes,NaN,__10000__,Low_spent_Small_value_payments,__-333333333333333333333333333__,Standard
freq,1,8,12384,44,2831.0,5565,7055,16,NaN,NaN,...,36479,24,NaN,446,52326,NaN,4305,25487,9,53174
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4194.071131,17.104395,...,NaN,NaN,32.285173,NaN,NaN,1403.118217,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3183.605038,117.454091,...,NaN,NaN,5.116875,NaN,NaN,8306.041270,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,303.645417,-1.000000,...,NaN,NaN,20.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1625.597917,3.000000,...,NaN,NaN,28.052567,NaN,NaN,30.306660,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3093.745000,6.000000,...,NaN,NaN,32.305784,NaN,NaN,69.249473,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5957.448333,7.000000,...,NaN,NaN,36.496663,NaN,NaN,161.224249,NaN,NaN,NaN,NaN


In [55]:
# Checking the logic and information of each of the variables

# Why do we have 100k customer IDs and just 12500 customerIDs?

# dataset.groupby("Customer_ID")["ID"].count().describe() 
# With this line of code we discover that each Customer_ID has 8 corresponding IDs.


# Which information is aggregated by Customer_ID? To know this, we gorup variables by Customer_ID and evaluate whether the number of unique value is 1 or not.

variables_by_customer_id = []
variables_by_id = []
other_variables = []


for variable in dataset.drop("Customer_ID", axis = 1).columns:
    summary_stats = dataset.groupby("Customer_ID")[variable].nunique().describe()
    if summary_stats["mean"] == 1:
        variables_by_customer_id.append(variable)
    elif summary_stats["mean"] == 8:
        variables_by_id.append(variable)
    else:
        other_variables.append(variable)

print(f'''

Variables by Customer_ID: {variables_by_customer_id}
Variables by ID: {variables_by_id}
Other variables: {other_variables}

      ''')









# If each ID corresponds to an specific product and this information is real, it means that each customer really has 8 products
# What value does month bring in? Month of what?
# dataset["Month"].unique()



Variables by Customer_ID: ['Name']
Variables by ID: ['ID', 'Month', 'Credit_Utilization_Ratio']
Other variables: ['Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score']

      


In [57]:
variables_by_customer_id = []
variables_by_id = []
other_variables = []

for variable in dataset.drop("Customer_ID", axis = 1).columns:
    dataset_cleaned = dataset.dropna(subset=[variable])
    summary_stats = dataset_cleaned.groupby("Customer_ID")[variable].nunique().describe()
    top_5_frequent = dataset_cleaned[variable].value_counts().head(5)
    least_5_frequent = dataset_cleaned[variable].value_counts().tail(5)
    print(summary_stats)
    print(f'''
          Top: {top_5_frequent},
          Least: {least_5_frequent}
          ''')
    if dataset.groupby("Customer_ID")[variable].nunique().describe()["mean"] == 1:
        variables_by_customer_id.append(variable)
    elif dataset.groupby("Customer_ID")[variable].nunique().describe()["mean"] == 8:
        variables_by_id.append(variable)
    else:
        other_variables.append(variable)
    


count    12500.0
mean         8.0
std          0.0
min          8.0
25%          8.0
50%          8.0
75%          8.0
max          8.0
Name: ID, dtype: float64

          Top: ID
0x1602     1
0x19c88    1
0x19caa    1
0x19ca5    1
0x19ca4    1
Name: count, dtype: int64,
          Least: ID
0xd94d     1
0xd94c     1
0xd94b     1
0xd94a     1
0x25fed    1
Name: count, dtype: int64
          
count    12500.0
mean         8.0
std          0.0
min          8.0
25%          8.0
50%          8.0
75%          8.0
max          8.0
Name: Month, dtype: float64

          Top: Month
August      12384
February    12376
May         12375
March       12373
April       12372
Name: count, dtype: int64,
          Least: Month
MarcE      1
ADgust     1
Say        1
Marcm      1
Jonuary    1
Name: count, dtype: int64
          
count    12500.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: Name, dtype: float64

          Top:

In [ ]:
# Are there duplicated values in the dataset?

dataset.duplicated().sum()

# No duplicated rows

0